In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_excel("2020_05_05_CAD_PRS_267-converted.xlsx")

In [4]:
df.head()

,rsID,Position,Effect\nAllele,P-value,Gene,Weight,r2,Genotyping,LD
0,rs2843152,1:2245570,C,0.000013,SKI,0.04240,0.78639,Imputed,NaN
1,rs35465346,1:22132518,G,0.000009,LDLRAD2,0.05465,0.84007,Imputed,NaN
2,rs7538207,1:22310674,C,0.000010,CELA3B,0.09848,0.84791,Imputed,NaN
3,rs11811081,1:32026584,C,0.000061,TINAGL1,0.07660,0.7599,Imputed,NaN
4,rs12733730,1:37023446,A,0.000011,CSF3R,0.04601,0.7836,Imputed,NaN


In [5]:
Initial_df = df[["rsID","Position","Effect\nAllele","Weight"]].copy()

In [6]:
Initial_df.head()

,rsID,Position,Effect\nAllele,Weight
0,rs2843152,1:2245570,C,0.04240
1,rs35465346,1:22132518,G,0.05465
2,rs7538207,1:22310674,C,0.09848
3,rs11811081,1:32026584,C,0.07660
4,rs12733730,1:37023446,A,0.04601


In [7]:
Chromosome = df["Position"].str.split(":").str[0]

In [8]:
PositionNum = df["Position"].str.split(":").str[-1]

In [9]:
Initial_df["NonVCFChromosome"] = Chromosome
Initial_df["NonVCFPositionNum"] = PositionNum

In [10]:
Initial_df= Initial_df.rename(columns= {"Effect\nAllele": "EffectAllele"})

In [11]:
Initial_df.head()

,rsID,Position,EffectAllele,Weight,NonVCFChromosome,NonVCFPositionNum
0,rs2843152,1:2245570,C,0.04240,1,2245570
1,rs35465346,1:22132518,G,0.05465,1,22132518
2,rs7538207,1:22310674,C,0.09848,1,22310674
3,rs11811081,1:32026584,C,0.07660,1,32026584
4,rs12733730,1:37023446,A,0.04601,1,37023446


In [12]:
import allel

In [82]:
%%time
callset = allel.read_vcf('xaa.vcf')

CPU times: user 1min 1s, sys: 11.7 s, total: 1min 12s
Wall time: 1min 18s


In [83]:
%%time
sorted(callset.keys())

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 9.06 µs


['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [84]:
%%time
genotype = callset['calldata/GT']

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [85]:
%%time
genotype.shape

CPU times: user 5 µs, sys: 12 µs, total: 17 µs
Wall time: 20 µs


(42999961, 1, 2)

In [86]:
%%time
genotype # convert 3d to 1d how to remove extra dimension of numpy array

CPU times: user 3 µs, sys: 9 µs, total: 12 µs
Wall time: 14.1 µs


array([[[0, 1]],

       [[0, 0]],

       [[1, 1]],

       ...,

       [[0, 0]],

       [[0, 0]],

       [[0, 0]]], dtype=int8)

In [87]:
%%time
array = genotype.squeeze()

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 10.7 µs


In [88]:
%%time
VCF_df= pd.DataFrame.from_records(array)

CPU times: user 1min 4s, sys: 26.5 s, total: 1min 30s
Wall time: 1min 46s


In [89]:
%%time
VCF_df["Genotype"] = VCF_df[0].astype(str) + "/" + VCF_df[1].astype(str)

CPU times: user 52.9 s, sys: 22.2 s, total: 1min 15s
Wall time: 1min 24s


In [90]:
%%time
VCF_df.head()

CPU times: user 382 µs, sys: 1.23 ms, total: 1.61 ms
Wall time: 2.72 ms


,0,1,Genotype
0,0,1,0/1
1,0,0,0/0
2,1,1,1/1
3,0,0,0/0
4,0,0,0/0


In [91]:
%%time
del VCF_df[1]
del VCF_df[0]

CPU times: user 133 ms, sys: 245 ms, total: 378 ms
Wall time: 424 ms


In [92]:
%%time
VCF_df

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


,Genotype
0,0/1
1,0/0
2,1/1
3,0/0
4,0/0
...,...
42999956,0/0
42999957,0/0
42999958,0/0
42999959,0/0


In [93]:
%%time
VCFChrom = callset['variants/CHROM']
VCFPos = callset['variants/POS']
VCFid = callset['variants/ID']
VCFRef = callset['variants/REF']
VCFAlt = callset['variants/ALT'][:,0]

CPU times: user 1.66 s, sys: 9 s, total: 10.7 s
Wall time: 17.1 s


In [94]:
%%time
VCF_df["VCFChrom"] = VCFChrom
VCF_df["VCFPos"] = VCFPos
VCF_df["VCFid"] = VCFid
VCF_df["VCFRef"] = VCFRef
VCF_df["VCFAlt"] = VCFAlt

CPU times: user 2.24 s, sys: 10.1 s, total: 12.3 s
Wall time: 19.4 s


In [95]:
%%time
VCF_df.head()

CPU times: user 1.07 ms, sys: 4.18 ms, total: 5.24 ms
Wall time: 10.3 ms


,Genotype,VCFChrom,VCFPos,VCFid,VCFRef,VCFAlt
0,0/1,1,10177,rs367896724,A,AC
1,0/0,1,10235,rs540431307,T,TA
2,1/1,1,10352,rs555500075,T,TA
3,0/0,1,10505,rs548419688,A,T
4,0/0,1,10506,rs568405545,C,G


In [96]:
%%time
Initial_df.head()

CPU times: user 282 µs, sys: 966 µs, total: 1.25 ms
Wall time: 2.96 ms


,rsID,Position,EffectAllele,Weight,NonVCFChromosome,NonVCFPositionNum
0,rs10109493,1:2245570,C,0.04240,1,2245570
1,rs10139550,1:22132518,G,0.05465,1,22132518
2,rs10168194,1:22310674,C,0.09848,1,22310674
3,rs10305649,1:32026584,C,0.07660,1,32026584
4,rs10417115,1:37023446,A,0.04601,1,37023446


In [97]:
%%time
#Start of analyzing part
final_df = pd.DataFrame()
final_df["Position"] = Initial_df["Position"]
final_df

CPU times: user 4.95 ms, sys: 5.93 ms, total: 10.9 ms
Wall time: 17 ms


,Position
0,1:2245570
1,1:22132518
2,1:22310674
3,1:32026584
4,1:37023446
...,...
262,21:40054149
263,22:19514013
264,22:19960184
265,22:24575333


In [98]:
import re

In [101]:
%%time
rs_id = Initial_df["rsID"].values
rs_idlist = rs_id.tolist()
sorted(rs_idlist)



CPU times: user 62 µs, sys: 1e+03 ns, total: 63 µs
Wall time: 66 µs


['rs10109493',
 'rs10139550',
 'rs10168194',
 'rs10305649',
 'rs10417115',
 'rs10423964',
 'rs10455872',
 'rs10513507',
 'rs10774625',
 'rs10818583',
 'rs10840293',
 'rs10857147',
 'rs1088868',
 'rs10890013',
 'rs10929113',
 'rs10955380',
 'rs11057401',
 'rs11057830',
 'rs11077501',
 'rs11115214',
 'rs111245230',
 'rs11153071',
 'rs11170820',
 'rs11172113',
 'rs111777100',
 'rs11191416',
 'rs1122326',
 'rs112370447',
 'rs11257613',
 'rs112635299',
 'rs112941079',
 'rs113025579',
 'rs113148244',
 'rs113348108',
 'rs113832197',
 'rs114123510',
 'rs11485595',
 'rs11509880',
 'rs11552449',
 'rs11556924',
 'rs11591147',
 'rs116082507',
 'rs11617955',
 'rs11654510',
 'rs116843064',
 'rs117113213',
 'rs11728590',
 'rs117592425',
 'rs117696200',
 'rs117938894',
 'rs11810571',
 'rs11811081',
 'rs11838776',
 'rs11955380',
 'rs12146487',
 'rs1214752',
 'rs12202017',
 'rs12252333',
 'rs12485143',
 'rs12493885',
 'rs1250229',
 'rs12619842',
 'rs12733730',
 'rs12801636',
 'rs12891473',
 'rs12922',
 

In [102]:
%%time
VCF_df = VCF_df[VCF_df.VCFChrom != "X"]

CPU times: user 13.9 s, sys: 42.5 s, total: 56.4 s
Wall time: 1min 22s


In [103]:
VCF_df

,Genotype,VCFChrom,VCFPos,VCFid,VCFRef,VCFAlt
0,0/1,1,10177,rs367896724,A,AC
1,0/0,1,10235,rs540431307,T,TA
2,1/1,1,10352,rs555500075,T,TA
3,0/0,1,10505,rs548419688,A,T
4,0/0,1,10506,rs568405545,C,G
...,...,...,...,...,...,...
42999956,0/0,8,80203909,rs529586325,T,C
42999957,0/0,8,80203944,rs547698871,G,T
42999958,0/0,8,80203958,rs143594860,G,C
42999959,0/0,8,80203962,rs539567118,G,T


In [104]:
Initial_df

,rsID,Position,EffectAllele,Weight,NonVCFChromosome,NonVCFPositionNum
0,rs10109493,1:2245570,C,0.04240,1,2245570
1,rs10139550,1:22132518,G,0.05465,1,22132518
2,rs10168194,1:22310674,C,0.09848,1,22310674
3,rs10305649,1:32026584,C,0.07660,1,32026584
4,rs10417115,1:37023446,A,0.04601,1,37023446
...,...,...,...,...,...,...
262,rs964184,21:40054149,T,0.03312,21,40054149
263,rs9869263,22:19514013,A,0.06006,22,19514013
264,rs9897596,22:19960184,G,0.03938,22,19960184
265,rs9929108,22:24575333,C,0.06735,22,24575333


In [125]:
VCF_id = VCF_df["VCFid"].values
sorted(VCF_id)

['.;esv3585028',
 '.;esv3585028',
 '.;esv3585319',
 '.;esv3585319',
 '.;esv3585524',
 '.;esv3585524',
 '.;esv3585587',
 '.;esv3585587',
 '.;esv3585621',
 '.;esv3585621',
 '.;esv3585885',
 '.;esv3585885',
 '.;esv3587493',
 '.;esv3587493',
 '.;esv3588592',
 '.;esv3588592',
 '.;esv3591962;esv3591963',
 '.;esv3591962;esv3591963',
 '.;esv3591962;esv3591963',
 '.;esv3593789',
 '.;esv3593789',
 '.;esv3593987',
 '.;esv3593987',
 '.;esv3594072',
 '.;esv3594072',
 '.;esv3594380',
 '.;esv3594380',
 '.;esv3594832',
 '.;esv3594832',
 '.;esv3596086',
 '.;esv3596086',
 '.;esv3599414',
 '.;esv3599414',
 '.;esv3600652',
 '.;esv3600652',
 '.;esv3601194',
 '.;esv3601194',
 '.;esv3601602',
 '.;esv3601602',
 '.;esv3601892',
 '.;esv3601892',
 '.;esv3602005',
 '.;esv3602005',
 '.;esv3602998',
 '.;esv3602998',
 '.;esv3604433',
 '.;esv3604433',
 '.;esv3605221',
 '.;esv3605221',
 '.;esv3605236',
 '.;esv3605236',
 '.;esv3607736',
 '.;esv3607736',
 '.;esv3609074',
 '.;esv3609074',
 '.;esv3610190',
 '.;esv3610190'

In [124]:
no_idlist = []
for i in VCF_id:
    if not i.startswith("r") :
        VCF_id = VCF_id[VCF_id != i]
        no_idlist.append(i)


KeyboardInterrupt: 

['.;esv3585028',
 '.;esv3585028',
 '.;esv3585319',
 '.;esv3585319',
 '.;esv3585524',
 '.;esv3585524',
 '.;esv3585587',
 '.;esv3585587',
 '.;esv3585621',
 '.;esv3585621',
 '.;esv3585885',
 '.;esv3585885',
 '.;esv3587493',
 '.;esv3587493',
 '.;esv3588592',
 '.;esv3588592',
 '.;esv3591962;esv3591963',
 '.;esv3591962;esv3591963',
 '.;esv3591962;esv3591963',
 '.;esv3593789',
 '.;esv3593789',
 '.;esv3593987',
 '.;esv3593987',
 '.;esv3594072',
 '.;esv3594072',
 '.;esv3594380',
 '.;esv3594380',
 '.;esv3594832',
 '.;esv3594832',
 '.;esv3596086',
 '.;esv3596086',
 '.;esv3599414',
 '.;esv3599414',
 '.;esv3600652',
 '.;esv3600652',
 '.;esv3601194',
 '.;esv3601194',
 '.;esv3601602',
 '.;esv3601602',
 '.;esv3601892',
 '.;esv3601892',
 '.;esv3602005',
 '.;esv3602005',
 '.;esv3602998',
 '.;esv3602998',
 '.;esv3604433',
 '.;esv3604433',
 '.;esv3605221',
 '.;esv3605221',
 '.;esv3605236',
 '.;esv3605236',
 '.;esv3607736',
 '.;esv3607736',
 '.;esv3609074',
 '.;esv3609074',
 '.;esv3610190',
 '.;esv3610190'

In [ ]:


# %%time



# rs_id = Initial_df["rsID"].values

# found_list = []

# found_df = pd.DataFrame()
# missing_df = pd.DataFrame()

# for i in rs_id:
#     pattern = (r'\b{i}\b').format(i=i)
#     p = re.compile(pattern, flags=re.IGNORECASE)
    
#     if not VCF_df[[bool(p.search(x)) for x in VCF_df['VCFid']]].empty:
#         found_list.append("found")
#         result = VCF_df[[bool(p.search(x)) for x in VCF_df['VCFid']]]
#         VCFfound_df = VCF_df[result]
#         print(VCFfound_df)
# #         else:
# #             found_list.append("missing")
# #             missing_df = missing_df.append(result)


# # final_df["Found?"] = found_list
# # found_df.reset_index(inplace=True)
# missing_df.reset_index(inplace=True)

In [ ]:
%%time
found_df.drop("index", axis=1, inplace=True)

In [ ]:
%%time
missing_df.drop("index", axis=1, inplace=True)

In [ ]:
missing_df

In [ ]:
%%time
missing_df.index +=62

In [ ]:
missing_df

In [ ]:
found_df

In [ ]:
final_df 

In [ ]:
%%time
final_df = pd.concat([final_df, found_df], axis=1)

In [ ]:
final_df

In [ ]:
final_df["EffectAllele"] = Initial_df["EffectAllele"]

In [ ]:
final_df["Weight"] = Initial_df["Weight"]

In [ ]:
final_df

In [ ]:
%%time
score = []
for index, row in final_df.iterrows():
    if row["Genotype"] == "1/1":
        if row["VCFRef"] == row["EffectAllele"]:
            score.append(0)
        elif row["VCFAlt"] == row["EffectAllele"]:
            score.append(2)
    elif row["Genotype"] == "0/0":
        if row["VCFRef"] == row["EffectAllele"]:
            score.append(2)
        elif row["VCFAlt"] == row["EffectAllele"]:
            score.append(0)
    elif row["Genotype"] == "0/1":
        score.append(1)
    else:
        score.append(None)

        

In [ ]:
%%time
final_df["Score"] = pd.Series(score)

In [ ]:
final_df

In [ ]:
%%time
final_df["WeightedScore"] = final_df["Score"] * final_df["Weight"]

In [ ]:
final_df

In [ ]:
%%time
Observed_Score_Sum = final_df["Score"].sum()
Maximum_Expected_Weight = final_df["Weight"].sum()
Observed_Weighted_Sum = final_df["WeightedScore"].sum()
Maximum_Expected_Score_Sum = len(final_df)*2
Final_weighted_GRS = Observed_Weighted_Sum/Maximum_Expected_Weight
Final_unweighted_GRS = Observed_Score_Sum/Maximum_Expected_Score_Sum
print(Final_weighted_GRS)
print(Final_unweighted_GRS)